<p><font size="7" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="6" color='grey'> <b>
Modul 06: LangChain: Datenextraktion
</b></font> </br></p>


---

# 1 | Übersicht
---

# 5.1: Strukturierter Ausgabeparser

LangChain bietet eine breite Palette von Ausgabeparsern, die darauf ausgelegt sind, Informationen aus den von großen Sprachmodellen (LLMs) zurückgegebenen Ausgaben effizient zu extrahieren und zu strukturieren. Diese Ausgabeparser spielen eine entscheidende Rolle in der Architektur von LangChain und bilden in der Regel integrale Komponenten der sogenannten LangChain-Ketten. Diese Ketten sind konfigurierbare Operationssequenzen, die Modellausgaben verarbeiten und mit ihnen interagieren, um komplexe Aufgaben auszuführen. Um die Erstellung und Verwaltung dieser Ketten zu erleichtern, führt LangChain die LangChain Expression Language (LCEL) ein.

Als Nächstes werden wir uns damit befassen, wie LCEL die erweiterte Erstellung und Ausführung von LangChain-Ketten ermöglicht, und anschließend untersuchen, wie verschiedene Ausgabeparser innerhalb dieser Ketten genutzt werden können, um aus den LLM-Ausgaben präzise und umsetzbare Erkenntnisse zu gewinnen.

## LangChain Expression Language oder LCEL

LangChain Expression Language oder [LCEL](https://python.langchain.com/docs/expression_language/) bietet eine deklarative Möglichkeit, Ketten einfach zusammenzustellen. Von Anfang an hat LCEL die Überführung von Prototypen in die Produktion ohne Codeänderungen unterstützt. Dies umfasst alles von einfachen „Prompt + LLM“-Ketten bis hin zu hochkomplexen Ketten, die einige Benutzer erfolgreich mit Hunderten von Schritten in Produktionsumgebungen implementiert haben. Hier sind einige Gründe, warum Sie sich für LCEL entscheiden könnten:

* [First-class streaming support](https://python.langchain.com/docs/expression_language/streaming/): Indem Sie Ihre Ketten mit LCEL erstellen, erreichen Sie die optimale Zeit bis zum ersten Token, also die Zeit, die vergeht, bis der erste Ausgabeblock erscheint. In einigen Fällen bedeutet dies, Token direkt von einem LLM an einen Streaming-Ausgabeparser zu streamen, der analysierte, inkrementelle Ausgabeblöcke mit derselben Geschwindigkeit liefert, mit der der LLM-Anbieter die Rohtoken freigibt.

* [Async support](https://python.langchain.com/docs/expression_language/interface/): Mit LCEL erstellte Ketten können sowohl synchrone APIs (z. B. in Ihrem Jupyter-Notebook während der Prototyperstellung) als auch asynchrone APIs (z. B. in einem LangServe-Server) verwenden. Diese doppelte Fähigkeit ermöglicht es, dass derselbe Code sowohl in Prototypen als auch in der Produktion effizient ausgeführt wird und viele gleichzeitige Anfragen auf demselben Server verarbeitet.

* [Optimized parallel execution](https://python.langchain.com/docs/expression_language/primitives/parallel/): LCEL führt automatisch parallelisierbare Schritte in Ihren Ketten gleichzeitig aus, unabhängig davon, ob Sie synchrone oder asynchrone Schnittstellen verwenden, und minimiert so die Latenz.

* [Retries and fallbacks](https://python.langchain.com/docs/guides/productionization/fallbacks/): Sie können Wiederholungsversuche und Fallbacks für jeden Teil Ihrer LCEL-Kette konfigurieren und so die Zuverlässigkeit im großen Maßstab verbessern. Wir verbessern auch die Streaming-Unterstützung für Wiederholungsversuche und Fallbacks, um die Zuverlässigkeit zu verbessern, ohne die Latenz zu erhöhen.

* [Access to intermediate results](https://python.langchain.com/docs/expression_language/interface/#async-stream-events-beta): Bei komplexeren Ketten kann der Zugriff auf Zwischenergebnisse entscheidend sein. Mit dieser Funktion können Endbenutzer den Fortschritt sehen oder Entwickler beim Debuggen der Kette unterstützen. Zwischenergebnisse sind streambar und auf jedem LangServe-Server verfügbar.

* [Input and output schemas](https://python.langchain.com/docs/expression_language/interface/#input-schema): Jede LCEL-Kette enthält Pydantic- und JSONSchema-Schemata, die aus der Struktur Ihrer Kette abgeleitet werden. Diese Schemata sind für die Validierung von Ein- und Ausgaben unerlässlich und stellen eine Kernfunktion von LangServe dar.

* [Seamless LangSmith tracing](https://python.langchain.com/docs/langsmith/): Da Ketten immer komplexer werden, ist es wichtig, jeden Schritt nachzuverfolgen. LCEL protokolliert alle Schritte automatisch in LangSmith, um die Beobachtung und Debugging zu verbessern.

* [Seamless LangServe deployment](https://python.langchain.com/docs/langserve/): Die Bereitstellung einer mit LCEL erstellten Kette ist mit LangServe unkompliziert und ermöglicht nahtlose Übergänge von der Entwicklung zur Produktion.

# Einführung in den StructuredOutputParser

LangChain bietet eine Vielzahl von Tools, die dabei helfen, Informationen aus der Ausgabe großer Sprachmodelle (LLMs) zu verarbeiten und zu extrahieren. In diesem Abschnitt werden wir die Funktionen des Structured Output Parser untersuchen, der besonders nützlich ist, wenn Sie Informationen aus mehreren Feldern zurückgeben müssen. Dieser Parser ist gut darin, Modellausgaben in unterschiedliche Kategorien zu organisieren und zu segmentieren, wodurch die Daten besser handhabbar und interpretierbar werden. Obwohl der Pydantic/JSON-Parser eine robustere und funktionsreichere Option für die Handhabung komplexer Datenstrukturen bietet, ist der Structured Output Parser eine ausgezeichnete Wahl für Umgebungen mit begrenzten Rechenressourcen oder bei Verwendung weniger leistungsstarker Modelle. Er bietet eine unkomplizierte und effiziente Möglichkeit, die Ausgabe zu strukturieren, ohne das Modell oder das System zu überfordern.

Wir beginnen mit der Erstellung eines ResponseSchemas für jeden Wert, den wir extrahieren möchten. Wir müssen jeden Wert beschreiben. Wir konstruieren den StructuredOutputParser aus einer Liste dieser Schemata.

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(
        name="source",
        description="source used to answer the user's question, should be a website.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

Wir erstellen nun eine Eingabeaufforderungsvorlage, in der sowohl die Frage als auch die Formatierungsanweisungen angegeben werden können. Die soeben erstellten Schemata generieren die Formatierungsanweisungen.

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

Bevor wir das LLM abfragen, stellen wir eine Frage und sehen uns an, wie LangChang die Eingabeaufforderung erstellt.

In [ ]:
question = "When was the Python programming language introduced?"

print(prompt.invoke(question).text)

answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // answer to the user's question
	"source": string  // source used to answer the user's question, should be a website.
}
```
When was the Python programming language introduced?


Wie Sie sehen, verwendet LangChain die Schemata, um ein JSON-Format für die Rückgabe der Antwort anzugeben. Mit der Antwort in diesem JSON-Format ist es unkompliziert, die einzelnen Werte zu analysieren.

Wir konstruieren nun eine Kette, um den StructuredOutputParser zu verwenden und das LLM abzufragen.

In [ ]:
MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Initialisieren Sie das OpenAI LLM mit Ihrem API-Schlüssel
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)
chain = prompt | llm | output_parser

Nun präsentieren wir eine Frage, eine Antwort und eine Quelle für die Frage.

In [ ]:
result = chain.invoke({"question": question})
print(result)

{'answer': 'Python programming language was introduced in 1991.', 'source': 'https://en.wikipedia.org/wiki/Python_(programming_language)'}


## Erkennen und Übersetzen

Wir versuchen es jetzt mit einem Beispiel mit mehr Werten. Das folgende Programm akzeptiert Text in jeder Sprache und übersetzt ihn ins Französische, Spanische und Chinesische.

In [ ]:
response_schemas = [
    ResponseSchema(name="detected", description="The language of the user's input"),
    ResponseSchema(name="spanish", description="Spanish translation"),
    ResponseSchema(name="french", description="French translation"),
    ResponseSchema(name="chinese", description="Chinese translation"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="translate into the requested languages.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | llm | output_parser

Wir beginnen mit dem Ausprobieren eines englischen Satzes und beobachten, wie er in die anderen drei Sprachen übersetzt wird.

In [ ]:
question = "When was the Python programming language introduced?"
result = chain.invoke({"question": question})
print(result)

{'detected': 'en', 'spanish': '¿Cuándo se introdujo el lenguaje de programación Python?', 'french': 'Quand le langage de programmation Python a-t-il été introduit?', 'chinese': 'Python编程语言是什么时候推出的？'}


# 5.2: Andere Parser (Kommaliste, JSON, Pandas, Datetime)


In diesem Abschnitt werden wir untersuchen, wie LangChain vielseitige Parser bietet, die eine Vielzahl von Datenformaten verarbeiten können, was seine Funktionalität in zahlreichen Anwendungen verbessert. Unter anderem kann es nahtlos in Daten in Form von Pandas-Datenrahmen, kommagetrennten Listen, JSON-Strukturen und Datums-/Uhrzeitobjekten integriert werden. Diese Fähigkeit stellt sicher, dass sich LangChain an unterschiedliche Dateneingaben anpassen kann, was es zu einem leistungsstarken Werkzeug für die Datenmanipulation und -analyse in verschiedenen Kontexten macht. Wir werden einige dieser Parser näher betrachten und ihre praktischen Anwendungen demonstrieren und hervorheben, wie sie genutzt werden können, um Prozesse zu optimieren und aussagekräftige Erkenntnisse aus Daten zu gewinnen.

## Analysieren Sie die durch Kommas getrennte Listenantwort

Wir beginnen mit dem Parser „CommaSeparatedListOutputParser“, der die LLM-Ausgabe in einer durch Kommas getrennten Liste übernehmen und als Python-Liste extrahieren kann.

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List ten {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Initialisieren Sie das OpenAI LLM mit Ihrem API-Schlüssel
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

chain = prompt | llm | output_parser

Extrahieren Sie eine Liste von Städten.

In [ ]:
chain.invoke({"subject": "cities"})

['New York City',
 'Los Angeles',
 'Chicago',
 'Houston',
 'Phoenix',
 'Philadelphia',
 'San Antonio',
 'San Diego',
 'Dallas',
 'San Jose']

Extrahieren Sie eine Liste von Programmiersprachen.

In [ ]:
chain.invoke({"subject": "programming languages"})

['Java',
 'Python',
 'C++',
 'JavaScript',
 'Ruby',
 'Swift',
 'PHP',
 'C#',
 'Go',
 'Kotlin']

## JSON-Antwort analysieren

Wir können die Ausgabe des LLM in JSON formatieren. Für dieses Beispiel akzeptieren wir einen Satz, den wir als Englisch erkennen, und übersetzen ihn dann ins Spanische, Französische und Chinesische.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Definieren Sie Ihre gewünschte Datenstruktur.
class Translate(BaseModel):
  detected: str = Field(description="the detected language of the input")
  spanish: str = Field(description="the input translated to Spanish")
  french: str = Field(description="the input translated to French")
  chinese: str = Field(description="the input translated to Chinese")

# Und eine Abfrage, die ein Sprachmodell zum Auffüllen der Datenstruktur veranlassen soll.
input_text = "What is your name?"

# Richten Sie einen Parser ein und fügen Sie Anweisungen in die Eingabeaufforderungsvorlage ein.
parser = JsonOutputParser(pydantic_object=Translate)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{input}\n",
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"input": input_text})

{'detected': 'English',
 'spanish': '¿Cuál es tu nombre?',
 'french': 'Quel est ton nom?',
 'chinese': '你叫什么名字？'}

## Pandas-Dataframe abfragen

Zu den Funktionen von Langchain gehört das Parsen und Analysieren von Pandas-Datenrahmen mithilfe des PandasDataFrameOutputParser. Mit dieser Funktion können Benutzer in Pandas-Datenrahmen gespeicherte Daten nahtlos integrieren und mit Langchain diese Daten abfragen und daraus Erkenntnisse gewinnen. Durch die Nutzung des PandasDataFrameOutputParser kann Langchain die Struktur, den Inhalt und den Kontext des Datenrahmens interpretieren und so genaue Antworten auf Benutzeranfragen liefern. Diese Integration ist besonders nützlich für die Datenanalyse, da sie eine interaktivere und auf natürlicher Sprache basierende Untersuchung der in Pandas-Datenrahmen gespeicherten Daten ermöglicht.

Der folgende Code liest und zeigt die ersten Zeilen des klassischen [iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) an.

In [ ]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Laden Sie den Iris-Datensatz
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/iris.csv", na_values=["NA", "?"]
)

print(df.head())

   sepal_l  sepal_w  petal_l  petal_w      species
0      5.1      3.5      1.4      0.2  Iris-setosa
1      4.9      3.0      1.4      0.2  Iris-setosa
2      4.7      3.2      1.3      0.2  Iris-setosa
3      4.6      3.1      1.5      0.2  Iris-setosa
4      5.0      3.6      1.4      0.2  Iris-setosa


Als nächstes laden wir den Iris-Datenrahmen in eine PandasDataFrameOutputParser-Klasse.

In [ ]:
parser = PandasDataFrameOutputParser(dataframe=df)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

Wir fragen den Mittelwert einer der Spalten ab.

In [ ]:
query = "Get the mean of the sepal_l column."
parser_output = chain.invoke({"query": query})
print(parser_output)

{'mean': 5.843333333333334}


Wir fragen nach dem Unterelement einer der Spalten.

In [ ]:
query = "Get the sum of petal_w column."
parser_output = chain.invoke({"query": query})
print(parser_output)

{'sum': 179.90000000000003}


## Datum/Uhrzeit

Langchain enthält eine Funktion namens DatetimeOutputParser, die speziell zum Parsen von Datums- und Uhrzeitwerten aus Text entwickelt wurde. Diese Funktion ermöglicht es, in verschiedenen Formaten ausgedrückte Daten und Zeiten zu erkennen und zu interpretieren und sie in ein standardisiertes Datums- und Uhrzeitformat umzuwandeln. Diese Funktionalität ist von unschätzbarem Wert in Anwendungen, die Terminplanung, Datenanalyse oder jeden anderen Kontext beinhalten, in dem die genaue Handhabung von Daten und Zeiten unerlässlich ist. Durch die Verwendung des DatetimeOutputParser können Entwickler die Verarbeitung zeitlicher Daten optimieren und sicherstellen, dass ihre Anwendungen zeitbezogene Informationen effektiv verwalten und darauf reagieren können.

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

Wir können die Eingabeaufforderung anzeigen, die wir zum Abrufen von Daten verwenden werden.

In [ ]:
print(prompt)

input_variables=['question'] partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1327-02-03T20:56:56.489822Z, 1058-10-02T02:08:23.921844Z, 682-08-19T01:21:02.307266Z\n\nReturn ONLY this string, no other words!"} template='Answer the users question:\n\n{question}\n\n{format_instructions}'


Wir erstellen die Kette, die wir zum Parsen von Daten verwenden werden.

In [ ]:
chain = prompt | llm | output_parser

Wir werden nach zwei Daten fragen, einem realen und einem fiktiven.

In [ ]:
output = chain.invoke({"question": "When was the Python language introduced?"})
print(output)

1991-02-20 00:00:00


In [ ]:
output = chain.invoke({"question": "What is the date of the war in the video game Fallout?"})
print(output)

2077-10-23 08:00:00


# 5.3: Pydantic Parser

Pydantic ist eine Datenvalidierungs- und Einstellungsverwaltungsbibliothek in Python, die Python-Typanmerkungen verwendet. Sie ist so konzipiert, dass sie eine schnelle und einfache Datenanalyse und -validierung mithilfe des Standardtypisierungssystems von Python ermöglicht.

Einige der Hauptfunktionen von Pydantic:

* Datenvalidierung: Es validiert die Daten, um sicherzustellen, dass sie dem erwarteten Format entsprechen, und konvertiert Typen bei Bedarf.
* Editor-Unterstützung: Pydantic-Modelle sind Klassen, die die Typhinweise von Python nutzen und dadurch einfach mit modernen Editoren verwendet werden können, die Funktionen wie Typprüfung und Autovervollständigung bieten.
* Fehlerbehandlung: Es werden detaillierte und für Menschen lesbare Fehlerberichte bereitgestellt, um zu ermitteln, wo und warum die Datenvalidierung fehlgeschlagen ist.
* Einstellungsverwaltung: Pydantic wird häufig zum Verwalten von Einstellungen/Konfigurationen verwendet und erleichtert das Laden von Parametern aus Umgebungsvariablen, JSON-Dateien oder anderen Quellen.
* Erweiterbar: Sie können Modelle mit Methoden und Eigenschaften erweitern und die Validierungsdekoratoren von Pydantic verwenden, um benutzerdefinierte Validierungen durchzuführen.
* Integration mit anderen Bibliotheken: Es funktioniert gut mit vielen anderen Bibliotheken, wie z. B. FastAPI zum Erstellen von APIs, und verbessert deren Benutzerfreundlichkeit und Funktionalität.

Insgesamt wird Pydantic wegen seiner Robustheit und Benutzerfreundlichkeit beim Sicherstellen der Konformität der Dateneingaben mit angegebenen Formaten sehr geschätzt, was es zu einem wertvollen Werkzeug in der modernen Python-Entwicklung macht, insbesondere in der Webentwicklung und bei Datenverarbeitungsanwendungen.

LangChain, eine Bibliothek, die die Erstellung von Anwendungen mit Sprachmodellen erleichtern soll, bietet verschiedene Tools zur Verwaltung und Verbesserung der Interaktion mit diesen Modellen. Eines dieser Tools ist der PydanticOutputParser, der die leistungsstarken Validierungsfunktionen von Pydantic mit der Ausgabe großer Sprachmodelle (LLMs) wie GPT integriert.

Das Hauptziel des PydanticOutputParser besteht darin, sicherzustellen, dass die Ausgaben eines Sprachmodells strukturiert sind und einem vordefinierten Schema entsprechen. Dies ist insbesondere bei Anwendungen wichtig, bei denen konsistente und zuverlässige Datenformate entscheidend sind, z. B. bei Datenextraktionsaufgaben, API-Antworten oder jedem Szenario, das eine anschließende automatisierte Verarbeitung der Ausgabe des Modells erfordert.

In [ ]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Initialisieren Sie das OpenAI LLM mit Ihrem API-Schlüssel
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

Der folgende Code verwendet ein LLM, um einen Witz zu erzählen. Der pydantische Parser stellt sicher, dass der Witz mit einem Fragezeichen endet. Diese Überprüfung stellt sicher, dass das eher zufällige LLM eine Ausgabe erzeugt, die unseren Erwartungen entspricht.

In [ ]:
# Definieren Sie Ihre gewünschte Datenstruktur.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # Mit Pydantic können Sie ganz einfach benutzerdefinierte Validierungslogik hinzufügen.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Und eine Abfrage, die ein Sprachmodell zum Auffüllen der Datenstruktur veranlassen soll.
joke_query = "Tell me a joke about cats."

# Richten Sie einen Parser ein und fügen Sie Anweisungen in die Eingabeaufforderungsvorlage ein.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!')

In [ ]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")
    @validator('name')
    def validate_name(cls, value):
        parts = value.split()
        if len(parts) < 2:
            raise ValueError("Name must contain at least two words.")
        if not all(part[0].isupper() for part in parts):
            raise ValueError("Each word in the name must start with a capital letter.")
        return value

actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": actor_query})

Actor(name='Tom Hanks', film_names=['Forrest Gump', 'Cast Away', 'Saving Private Ryan', 'Toy Story', 'The Green Mile'])

# 5.4: Benutzerdefinierte Ausgabeparser

In bestimmten Szenarien möchten Sie möglicherweise einen benutzerdefinierten Parser erstellen, um die Modellausgabe eindeutig zu formatieren.

Es gibt zwei Möglichkeiten, einen benutzerdefinierten Parser zu erstellen:

* Verwenden von **RunnableLambda** oder **RunnableGenerator** in LCEL – Dies ist für die meisten Fälle der empfohlene Ansatz.
* Erben von einer der Basisklassen zur Ausgabeanalyse – Dies ist die anspruchsvollere Methode.

Die Unterschiede zwischen diesen Ansätzen sind meist oberflächlicher Natur und bestehen hauptsächlich darin, welche Rückrufe ausgelöst werden (z. B. on_chain_start vs. on_parser_start) und wie ein ausführbares Lambda im Vergleich zu einem Parser in einer Tracing-Plattform wie LangSmith visualisiert wird.

Ich schlage vor, zum Parsen ausführbare Lambdas und ausführbare Generatoren zu verwenden.

Der folgende Code erstellt ein grundlegendes LLM-Modell zur Verwendung.



In [ ]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-4o-mini'
TEMPERATURE = 0.0

# Initialisieren Sie das OpenAI LLM mit Ihrem API-Schlüssel
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

In diesem Abschnitt erstellen wir einen einfachen Parser, der die Groß-/Kleinschreibung der Modellausgabe umkehrt.

Wenn das Modell beispielsweise „Hallo Welt“ ausgibt, wandelt der Parser es in „HALLO WELT“ um.

In [ ]:
from typing import Iterable

from langchain_core.messages import AIMessage, AIMessageChunk

def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()


chain = llm | parse
chain.invoke("hello")

'hELLO! hOW CAN i ASSIST YOU TODAY?'

## Erben aus Parsing-Basisklassen

Eine andere Möglichkeit, einen Parser zu implementieren, besteht in der Erbung von BaseOutputParser, BaseGenerationOutputParser oder einem anderen Basisparser, je nach Bedarf.

Für die meisten Anwendungsfälle empfehlen wir diesen Ansatz grundsätzlich nicht, da er mehr Code erfordert, ohne nennenswerte Vorteile zu bieten.

Der einfachste Typ eines Ausgabeparsers erweitert die Klasse BaseOutputParser und muss die folgenden Methoden implementieren:

* **parsen**: Nimmt die String-Ausgabe vom Modell und analysiert sie.
* **(optional) _type**: Identifiziert den Namen des Parsers.
Wenn die Ausgabe des Chat-Modells oder LLM fehlerhaft ist, kann der Parser eine OutputParserException auslösen, um anzuzeigen, dass die Analyse aufgrund einer fehlerhaften Eingabe fehlgeschlagen ist. Durch die Verwendung dieser Ausnahme kann Code, der den Parser verwendet, Ausnahmen konsistent behandeln.

Da BaseOutputParser die Runnable-Schnittstelle implementiert, wird jeder benutzerdefinierte Parser, den Sie auf diese Weise erstellen, zu einem gültigen LangChain Runnable und profitiert von automatischer asynchroner Unterstützung, Batch-Schnittstelle, Protokollierungsunterstützung und mehr.

Hier ist ein einfacher Parser, der eine Zeichenfolgendarstellung eines Booleschen Werts (z. B. JA oder NEIN) analysieren und in den entsprechenden Booleschen Typ konvertieren kann.

In [ ]:
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseOutputParser


class BooleanOutputParser(BaseOutputParser[bool]):
    """Custom parser to interpret 'YES'/'NO' strings as boolean values."""

    true_val: str = "YES"
    false_val: str = "NO"

    def parse(self, text: str) -> bool:
        """
        Parse the input text and return a boolean value.

        Args:
            text (str): The input text to parse.

        Returns:
            bool: True if text matches true_val, False if it matches false_val.

        Raises:
            OutputParserException: If the text does not match true_val or false_val.
        """
        cleaned_text = text.strip().upper()
        if cleaned_text not in (self.true_val.upper(), self.false_val.upper()):
            raise OutputParserException(
                f"BooleanOutputParser expected output value to be either "
                f"{self.true_val} or {self.false_val} (case-insensitive). "
                f"Received {cleaned_text}."
            )
        return cleaned_text == self.true_val.upper()

    @property
    def _type(self) -> str:
        """
        Return the type of the parser.

        Returns:
            str: The type of the parser.
        """
        return "boolean_output_parser"


In [ ]:
parser = BooleanOutputParser()
parser.invoke("YES")

True

In [ ]:
try:
    parser.invoke("MEOW")
except Exception as e:
print(f"Eine Ausnahme vom Typ {type(e)} wurde ausgelöst.")

Triggered an exception of type: <class 'langchain_core.exceptions.OutputParserException'>


In [ ]:
parser = BooleanOutputParser(true_val="OKAY")
parser.invoke("OKAY")

True

In [ ]:
parser.batch(["OKAY", "NO"])

[True, False]

In [ ]:
await parser.abatch(["OKAY", "NO"])

[True, False]

In [ ]:
llm.invoke("say either OKAY or NO")

AIMessage(content='OKAY', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 13, 'total_tokens': 15}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b969e2cc-df17-4c98-a890-703318c5a4c2-0')

In [ ]:
chain = llm | parser
chain.invoke("say either OKAY or NO")

True

### Entfernen von Nicht-Python-Text

Large Language Models (LLMs) wie GPT-4 können Text generieren, der Code und erklärende Beschreibungen nahtlos vermischt. Dies kann zwar für Lern- und Dokumentationszwecke unglaublich nützlich sein, kann aber eine Herausforderung darstellen, wenn aus solchen Ausgaben mit gemischtem Inhalt nur der Code extrahiert und ausgeführt werden muss. Um dies zu beheben, implementieren wir eine einfache Funktion, die nicht-Python-Codezeilen aus einer gegebenen Textzeichenfolge entfernt.

Bei diesem Ansatz werden reguläre Ausdrücke verwendet, um Zeilen zu identifizieren und beizubehalten, die der typischen Python-Syntax entsprechen, während Zeilen verworfen werden, die beschreibender Text zu sein scheinen. Aufgrund der inhärenten Komplexität und Variabilität sowohl von Python-Code als auch von natürlicher Sprache kann diese Methode jedoch nie perfekt sein. Sie basiert auf heuristischen Mustern, die Code manchmal fälschlicherweise als Text klassifizieren oder umgekehrt.

Im nächsten Abschnitt werden wir untersuchen, wie ein anderes LLM beim Entfernen von Nicht-Python-Code helfen kann und möglicherweise eine ausgefeiltere und genauere Lösung bietet. Das folgende Beispiel enthält eine Mischung aus LLM-Kommentaren und generiertem Code.









In [ ]:
# Beispielverwendung
mixed_text = """
Yes, you can estimate the value of Pi using various methods in Python. One
common approach is the Monte Carlo method. Here's a simple example:

```python
import random

def estimate_pi(num_samples):
    inside_circle = 0

    for _ in range(num_samples):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        distance = x**2 + y**2

        if distance <= 1:
            inside_circle += 1

    pi_estimate = (inside_circle / num_samples) * 4
    return pi_estimate

num_samples = 1000000
pi_estimate = estimate_pi(num_samples)
print(f"Geschätzter Wert von Pi: {pi_estimate}")
```

This code uses the Monte Carlo method to estimate Pi by generating random points
within a unit square and checking how many fall inside a quarter circle. The
ratio of points inside the circle to the total points, multiplied by 4, gives an
estimate of Pi.

Would you like to explore other methods or need further explanation on this
approach?

"""

Wir bieten jetzt eine Funktion zum Entfernen des nicht-Python-Texts. Die Funktion extract_python_code verwendet reguläre Ausdrücke, um Python-Codeblöcke zu finden und zu extrahieren, die in drei Backticks eingeschlossen sind. Sie verwendet die Funktion re.findall mit einem Muster, das Text zwischen Python und Trennzeichen abgleicht. Das Flag re.DOTALL ist enthalten, um sicherzustellen, dass der reguläre Ausdruck mit Zeilenumbruchzeichen innerhalb des Codeblocks übereinstimmen kann, wodurch die Extraktion mehrzeiliger Codes ermöglicht wird. Die übereinstimmenden Codeblöcke werden dann zu einer einzigen Zeichenfolge zusammengefügt, wobei führende oder nachfolgende Leerzeichen mithilfe der Strip-Methode entfernt werden. Dieser Ansatz isoliert den Python-Code effektiv vom umgebenden gemischten Text und erleichtert so das Extrahieren und unabhängige Verwenden.

In [ ]:
import re

def extract_python_code(mixed_text):
    code_blocks = re.findall(r'```python(.*?)```', mixed_text, re.DOTALL)
    return "\n".join(code_blocks).strip()

Im Folgenden wird gezeigt, wie wir extract_python_code verwenden können, um den Python-Code zu extrahieren.



In [ ]:
python_code = extract_python_code(mixed_text)
print(python_code)

import random

def estimate_pi(num_samples):
    inside_circle = 0

    for _ in range(num_samples):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        distance = x**2 + y**2

        if distance <= 1:
            inside_circle += 1

    pi_estimate = (inside_circle / num_samples) * 4
    return pi_estimate

num_samples = 1000000
pi_estimate = estimate_pi(num_samples)
print(f"Estimated value of Pi: {pi_estimate}")


### Erstellen eines Code-Ausgabeparsers.

Wir erstellen jetzt einen benutzerdefinierten Ausgabeparser, um allen nicht-Python-Code zu entfernen.

In [ ]:
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseOutputParser

class CodeOutputParser(BaseOutputParser[str]):
    """Custom code parser."""

    def parse(self, text):
      return extract_python_code(text)

    @property
    def _type(self) -> str:
        return "CodeOutputParser"

Wie hier gezeigt, wird nur der Python-Code ausgegeben.

In [ ]:
from IPython.display import Code, display

parser = CodeOutputParser()
chain = llm | parser
result = chain.invoke("Can I create Python code to estimate the value of Pi.")
display(Code(result, language='python'))

import random

def estimate_pi(num_points):
    inside_circle = 0
    total_points = num_points

    for _ in range(num_points):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)

        if x**2 + y**2 <= 1:
            inside_circle += 1

    pi_estimate = 4 * inside_circle / total_points
    return pi_estimate

num_points = 1000000
pi_estimate = estimate_pi(num_points)
print(f"Estimated value of Pi using {num_points} points: {pi_estimate}")

# 5.5: Ausgabe-Fixing-Parser

Dieser Ausgabeparser umschließt einen anderen Ausgabeparser und ruft, falls der erste fehlschlägt, einen anderen LLM auf, um etwaige Fehler zu beheben.

Aber wir können auch andere Dinge tun, außer Fehler zu werfen. Insbesondere können wir die falsch formatierte Ausgabe zusammen mit den formatierten Anweisungen an das Modell übergeben und es bitten, sie zu korrigieren.

Für dieses Beispiel verwenden wir den oben genannten Pydantic-Ausgabeparser. Folgendes passiert, wenn wir ihm ein Ergebnis übergeben, das nicht dem Schema entspricht:



In [ ]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-4o-mini'
TEMPERATURE = 0.0

# Initialisieren Sie das OpenAI LLM mit Ihrem API-Schlüssel
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)


Der Ausgabekorrekturparser umschließt nicht nur einen anderen Parser, sondern ruft auch ein sekundäres Sprachmodell auf, um Fehler zu korrigieren, wenn der erste fehlschlägt. Dieser innovative Parser kann mehr als nur Fehler behandeln – er verarbeitet aktiv falsch formatierte Ausgaben und verwendet formatierte Anweisungen, um das Modell um Korrekturen zu bitten.

In diesem Szenario demonstrieren wir die Fähigkeiten eines Pydantic-Ausgabeparsers. Folgendes passiert, wenn wir ihm ein Ergebnis übergeben, das nicht dem Schema entspricht:

In [ ]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

Wir simulieren jetzt eine falsch formatierte Antwort.

In [ ]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

Wir überprüfen, ob tatsächlich eine Ausnahme ausgelöst wird.

In [ ]:
try:
    parser.parse(misformatted)
except Exception as e:
print(f"Eine Ausnahme vom Typ {type(e)} wurde ausgelöst.")

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}

Jetzt nutzen wir den OutputFixingParser, um das Problem mithilfe eines LLM zu beheben.

In [ ]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

Wir sehen jetzt den richtig geladenen, falsch formatierten Text.

In [ ]:
new_parser.parse(misformatted)

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

Beachten Sie, dass der Zielparser den Methodenaufruf **get_format_instructions** unterstützen muss, damit der Formatkorrekturparser funktioniert, um eine Zeichenfolge zurückzugeben, die beschreibt, wie das Format angefordert wurde.

Wir stellen auch fest, dass es Swahili erkennen und übersetzen kann.

In [ ]:
question = "Sijui nini kinaendelea?"
result = chain.invoke({"question": question})
print(result)

{'detected': 'Swahili', 'spanish': '¿Qué está pasando?', 'french': "Qu'est-ce qui se passe?", 'chinese': '我不知道发生了什么？'}


# Modul 5 Aufgabe

Die erste Aufgabe findet ihr hier: [assignment 5](https://github.com/jeffheaton/app_generative_ai/blob/main/assignments/assignment_yourname_t81_559_class5.ipynb)